In [123]:
pretty_print(sage.misc.html.math_parse('Let $M$ be a module of rank 3 over the ring of 7-adic numbers with a variable precision of 40.'))
pretty_print(sage.misc.html.math_parse('We consider the quadratic form: $Q(x_{1},x_{2},x_{3}) = 2x_{1}^{2}+21x_{1}x_{2}+14x_{1}x_{3}+10x_{2}x_{1}+x_{2}^{2}+19x_{2}x_{3}+x_{3}x_{1}+18x_{3}x_{2}+71x_{3}x_{2}$.'))

# Initialization

R = Zp(7, prec = 40)                                                          
pi = R.uniformizer()
M = R^3
basis_elements = list(M.basis())
global normal_set
normal_set = []
coefficients_quadratic = [R(1),R(21),R(14),R(10),R(1),R(19),R(1),R(18),R(71)]

# returns a boolean
def is_not_zero(element):
    if element.valuation() > 40:
        return False
    else:
        return True
    
# returns the co-ordinate sum of two tuples
def add_elements(v1,v2):
    v = []
    for i in range(len(v1)):
        v.append(v1[i]+v2[i])
    return v

# scales a tuple by a number
def scale_element(alpha,v):
    w = []
    for i in range(len(v)):
        w.append(alpha * v[i])
    return w

# swaps two items in a list
def rearrange(lis,num1,num2):
    if num1 == num2:
        return lis
    else:
        new_list = []
        for i in range(0,len(lis)):
            if i == num1:
                new_list.append(lis[num2])
            elif i == num2:
                new_list.append(lis[num1])
            else:
                new_list.append(lis[i])
        return new_list

# computes a function
def quadratic_map(element):
    out = R(0)
    count = 0
    for i in range(3):
        for j in range(3):
            out += coefficients_quadratic[count]*element[i]*element[j]
            count += 1
    return out

# computes the induced function
def bilinear_map(element_one,element_two):
    return quadratic_map(add_elements(element_one,element_two)) - quadratic_map(element_one) - quadratic_map(element_two)

# recursive algorithm
def normalizer(basis):
    value_collection=[]                                               # computes values of bilinear map for all possible combinations
    num = len(basis)
    for i in range(num):
        for j in range(num):
            entry = bilinear_map(basis[i],basis[j])
            value_collection.append(entry)
    
    trigger = True                                                    # checks if the set is already normalized
    for i in range(len(value_collection)):
        if(is_not_zero(value_collection[i])):
            trigger = False
            break
    if trigger:
        normal_set.append(basis)
        return 
    
    equally_valued = [0]                                              # searchs for the elements with the smallest valuation
    least_valuation = value_collection[0].valuation()
    for i in range(1,len(value_collection)):
        current_valuation = value_collection[i].valuation()
        if least_valuation < current_valuation:
            equally_valued = [i]
            least_valuation = current_valuation
        elif least_valuation == current_valuation:
            equally_valued.append(i)
        else:
            continue
            
    flag = 0                                                          # picks the smallest pair (i,j) as prescribed by the algorithm
    left = num
    right = num
    for i in range(0,len(equally_valued)):
        current = equally_valued[i]
        tuple_one = current // num
        tuple_two = current % num
        if tuple_one == tuple_two:
            left = tuple_one
            right = tuple_two
            flag = 1
            normal_set.append(basis[left])
            break
        if tuple_one<left:
            left = tuple_one
            right = tuple_two
    if R(2).is_unit() and flag == 0:
        flag = 1
        normal_set.append(add_elements(basis[left],basis[right]))
        
    if flag == 1:                                                    # step three of the algorithm
        basis = rearrange(basis,0,left)
        new_basis = []
        n = normal_set[-1]
        T = bilinear_map(n,n)
        for i in range(1,num):
            scaled = scale_element(R(-1)*bilinear_map(n,scale_element(R(1)/T,basis[i])),n)
            new = add_elements(basis[i],scaled) 
            new_basis.append(new)
            
    elif flag == 0:                                                  # step four of the algorithm
        new_basis = []
        T = bilinear_map(basis[left],basis[right])
        T_valuation = T.valuation()
        exponent = R(1)/T
        for i in range(T_valuation):
            exponent*=pi
        normal_set.append(scale_element(exponent,basis[left]))
        normal_set.append(basis[right])
        basis = rearrange(basis,0,left)
        basis = rearrange(basis,1,right)
        f1 = normal_set[-2]
        f2 = normal_set[-1]
        T1 = bilinear_map(f1,f1)
        T2 = bilinear_map(f2,f2)
        T = bilinear_map(f1,f2)
        d  = T1*T2 - T*T
        for i in range(2,num):
            b = basis[i]
            t = T*bilinear_map(f2,b) - T2*bilinear_map(f1,b)
            u = T*bilinear_map(f1,b) - T1*bilinear_map(f2,b)
            scale1 = scale_element(t/d,f1)
            scale2 = scale_element(u/d,f2)
            new_basis.append(add_elements(b,add_elements(scale2,scale1)))
    
    total = len(new_basis)
    if total == 0:                                                  # returns the normalized basis
        return
    else:
        normalizer(new_basis)                                       # loops back with the new basis vectors
        
normalizer(basis_elements)

pretty_print(sage.misc.html.math_parse('Output of the algorithm :'))

@interact
def _(mode=slider(['hide','display'])):
    if mode == 'display':
        show(normal_set)
    else:
        print("")

pretty_print(sage.misc.html.math_parse('The normalized form of $Q$ is :'))
pretty_print(LatexExpr('<1>\perp<1591701440227256996435358784805761>\perp<232850785404196425810815287223503>'))

Let <script type="math/tex">M</script> be a module of rank 3 over the ring of 7-adic numbers with a variable precision of 40.

We consider the quadratic form: <script type="math/tex">Q(x_{1},x_{2},x_{3}) = 2x_{1}^{2}+21x_{1}x_{2}+14x_{1}x_{3}+10x_{2}x_{1}+x_{2}^{2}+19x_{2}x_{3}+x_{3}x_{1}+18x_{3}x_{2}+71x_{3}x_{2}</script>.

Output of the algorithm :

SW50ZXJhY3RpdmUgZnVuY3Rpb24gPGZ1bmN0aW9uIF8gYXQgMHg2ZmVjYTI4MzM5OD4gd2l0aCAxIHdpZGdldAogIG1vZGU6IFNlbGVjdGlvblNsaWRlcihkZXNjcmlwdGlvbj11J21vZGUnLCDigKY=


The normalized form of <script type="math/tex">Q</script> is :

<1>\perp<1591701440227256996435358784805761>\perp<232850785404196425810815287223503>